In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import random

In [2]:
# imports
import os
import pandas as pd
import numpy as np

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()

import nltk
#file for punkt splitter
nltk.download('punkt');
#file for vader sentiment
nltk.download('vader_lexicon');

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=20,20
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


<br><br>

## Data loading/collection

In [3]:
series_len = 200
mask_value = -10
max_len = 0

def text_to_sentiments(text):
    sentences = nltk.tokenize.sent_tokenize(text)
    scores = [ sia.polarity_scores(s)['compound'] for s in sentences] #list of compound score per sentence
    if len(scores) < series_len:
        t  = series_len - len(scores)
        scores = np.pad(scores, (t, 0), mode='constant', constant_values=mask_value)
    else:
        scores = scores[(-1*series_len):]
    return scores

In [4]:
try:
    df = pd.read_csv(os.path.join('out', 'truefake_series.csv'))
    print('Succesfully loaded data')
except FileNotFoundError:
    print('ERROR: Couldn\'t find data, will start building it now...')
    dataset_path_true = os.path.join("sources", "ISOT", "True.csv")
    dataset_path_fake = os.path.join("sources", "ISOT", "Fake.csv")

    dataset_load_true = pd.read_csv(dataset_path_true, encoding='utf-8') # make sure to use the right encoding
    dataset_load_fake = pd.read_csv(dataset_path_fake, encoding='utf-8') 
    
    df_T = pd.DataFrame(columns = ['scores', 'veracity'])
    df_F = pd.DataFrame(columns = ['scores', 'veracity'])

    df_T['scores'] = dataset_load_true.text.apply(text_to_sentiments)
    df_T['veracity'] = 'T'

    df_F['scores'] = dataset_load_fake.text.apply(text_to_sentiments)
    df_F['veracity'] = 'F'

    df = pd.concat([df_T,df_F], ignore_index=True)
    df = df[df['scores'].map(lambda d: len(d)) > 0]
    df.reset_index(drop = True, inplace=True)
    
    df = pd.concat([df_T,df_F], ignore_index=True)
    df = df[df['scores'].map(lambda d: len(d)) > 0]
    df.reset_index(drop = True, inplace=True)
    
    df.to_csv(os.path.join('out', 'truefake_series.csv'), index=False)
    
finally:
    display(df)

ERROR: Couldn't find data, will start building it now...


scores veracity
0      [-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10...        T
1      [-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10...        T
2      [-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10...        T
3      [-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10...        T
4      [-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10...        T
...                                                  ...      ...
44893  [-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10...        F
44894  [-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10...        F
44895  [-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10...        F
44896  [-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10...        F
44897  [-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10...        F

[44898 rows x 2 columns]

<br><br>
## Splitting data

In [55]:
def train_test_split(df_in, test_size, shuffle=True):
    
    if shuffle:
        df_in = df_in.sample(frac=1).reset_index(drop=True)
    
    mask_test = np.random.rand(len(df_in)) < test_size # the sample is large enough to probably not care about proper division
    
    train = df_in[~mask_test]
    test = df_in[mask_test]
    
    X_train = train['scores'].tolist()
    y_train = train['veracity'].tolist()
    
    X_test = test['scores'].tolist()
    y_test = test['veracity'].tolist()
    
    return (X_train, y_train), (X_test, y_test)

In [64]:
np.random.seed(160)
random.seed(160)

(X_train, y_train), (X_test, y_test) = train_test_split(df, test_size = 0.4)
print("train: ",len(X_train), len(y_train),
     "\ntest: ",len(X_test), len(y_test),
     "\ntest/total: {:0.3f}".format(len(X_test)/(len(X_test)+len(X_train)))
    )

train:  26964 26964 
test:  17934 17934 
test/total: 0.399


<br><br>

## Building the model

In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

In [6]:
model.add(
    Embedding(input_dim=series_len,
              input_length=series_len, #??
              output_dim=1 # True or False as a -1 to 1?
    
    )
)
model.add(Masking(mask_value=mask_value)) # 200 [series len] timesteps (sentences) with 1 feature (sentiment)

SyntaxError: invalid syntax (<ipython-input-6-dbb333efc81c>, line 4)